In [0]:
'''from google.colab import drive
drive.mount('/data/')
from pathlib import Path
base_dir = ('/data/My Drive')'''

"from google.colab import drive\ndrive.mount('/data/')\nfrom pathlib import Path\nbase_dir = ('/data/My Drive')"

In [0]:
#!cp  /data/My\ Drive/DeepLearning/student_data.zip /content
#!unzip student_data.zip

In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import sys

import os
import random

import numpy as np
import pandas as pd

import torchvision.transforms as transforms

import sys
from data_helper import UnlabeledDataset, LabeledDataset, ObjDetectionLabeledDataset
from helper import collate_fn, draw_box

import utils
from engine import train_one_epoch, evaluate

In [0]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

In [0]:
image_folder = 'data'
annotation_csv = 'data/annotation.csv'

In [0]:
# You shouldn't change the unlabeled_scene_index
# The first 106 scenes are unlabeled
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 120)

## validation scene index.
validation_scene_index = np.arange(120, 134)

In [0]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [0]:
# The labeled dataset can only be retrieved by sample.
# And all the returned data are tuple of tensors, since bounding boxes may have different size
# You can choose whether the loader returns the extra_info. It is optional. You don't have to use it.
labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
trainloader = torch.utils.data.DataLoader(labeled_trainset, batch_size=2, shuffle=True, num_workers=2,drop_last=True, collate_fn=collate_fn)


# In[ ]:


# The labeled dataset can only be retrieved by sample.
# And all the returned data are tuple of tensors, since bounding boxes may have different size
# You can choose whether the loader returns the extra_info. It is optional. You don't have to use it.
validation_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=validation_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
valLoader = torch.utils.data.DataLoader(validation_trainset, batch_size=2, shuffle=True, num_workers=2, drop_last=True, collate_fn=collate_fn)



TrainUpTrainset = ObjDetectionLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=True,
                                  batch_size = 2
                                 )
TrainLoaderNew = torch.utils.data.DataLoader(TrainUpTrainset, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)


validationUpTrainset = ObjDetectionLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=validation_scene_index,
                                  transform=transform,
                                  extra_info=True,
                                  batch_size = 2
                                 )
valLoaderNew = torch.utils.data.DataLoader(validationUpTrainset, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)



In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# import your model.

from modelobj import NetObj
mdl = NetObj()
model = mdl.model



#print(model)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [0]:
# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizerNew = torch.optim.SGD(params, lr=0.005,
                            momentum=0.09, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizerNew,
                                               step_size=3,
                                               gamma=0.1)

In [0]:
for epoch in range(0, 25):
  # train for one epoch, printing every 10 iterations
  train_one_epoch(model, optimizerNew, TrainLoaderNew, device, epoch, print_freq=10)
  # update the learning rate
  lr_scheduler.step()
  # evaluate on the test dataset
  evaluate(model, valLoaderNew, device=device)

  model_file = 'modelobj/fasterrcnn_model_' + str(epoch) + '.pth'
  torch.save({'modelObjectDetection_state_dict': model.state_dict()},model_file)
  print('\nSaved model to ' + model_file )

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [  0/882]  eta: 0:10:30  lr: 0.000011  loss: 3.1468 (3.1468)  loss_classifier: 2.1781 (2.1781)  loss_box_reg: 0.0056 (0.0056)  loss_objectness: 0.6916 (0.6916)  loss_rpn_box_reg: 0.2715 (0.2715)  time: 0.7144  data: 0.3012  max mem: 2469
Epoch: [0]  [ 10/882]  eta: 0:05:54  lr: 0.000067  loss: 3.1784 (4.9629)  loss_classifier: 2.1754 (2.1765)  loss_box_reg: 0.0168 (0.0172)  loss_objectness: 0.6923 (0.6925)  loss_rpn_box_reg: 0.3104 (2.0766)  time: 0.4064  data: 0.0351  max mem: 2733
Epoch: [0]  [ 20/882]  eta: 0:05:36  lr: 0.000124  loss: 3.3605 (4.2493)  loss_classifier: 2.1739 (2.1738)  loss_box_reg: 0.0142 (0.0144)  loss_objectness: 0.6923 (0.6924)  loss_rpn_box_reg: 0.4769 (1.3688)  time: 0.3746  data: 0.0088  max mem: 2733
Epoch: [0]  [ 30/882]  eta: 0:05:29  lr: 0.000181  loss: 3.3605 (3.9444)  loss_classifier: 2.1652 (2.1691)  loss_box_reg: 0.0101 (0.0164)  loss_objectness: 0.6924 (0.6924)  loss_rpn_box_reg: 0.4769 (1.0666)  time: 0.3757  data: 0.0096  max mem: 2733


In [0]:
#!pip install numpy==1.17.4

In [0]:
!cp -r modelobj /data/My\ Drive/DeepLearning/modelobj

#zip /content/modelobj.zip  /content/modelobj

In [0]:
#!cp  /data/My\ Drive/DeepLearning/student_data.zip /content
#!unzip student_data.zip